### 图像旋转需要用到一下核心技术：
- 1.旋转矩阵：旋转+平移
- 2.插值方法：1.最近邻  2.双线性（本块知识点用到的最多）  3.像素区域 。。。。。。。
- 3.边界填充
- 4.图片旋转，逆时针方向位正方向

In [2]:
# 导入 OpenCV 库
import cv2

# 1️⃣ 读取图像
# cv2.imread(filename, flags)
#  - filename: 图像路径
#  - flags: 读取模式，常见有
#     cv2.IMREAD_COLOR（默认）→ 彩色
#     cv2.IMREAD_GRAYSCALE → 灰度
#     cv2.IMREAD_UNCHANGED → 包含透明通道
image_np = cv2.imread("../image/image.jpg")

# 获取图像的形状信息（高度, 宽度, 通道数）
image_shape = image_np.shape  # 例如 (1080, 1920, 3)

# 2️⃣ 定义旋转参数
angle = 45  # 旋转角度（单位：度），正数为逆时针方向
scale = 0.5  # 缩放比例，小于1缩小，大于1放大

# 3️⃣ 获取旋转矩阵
# cv2.getRotationMatrix2D(center, angle, scale)
#  - center: 旋转中心 (x, y)
#  - angle: 旋转角度（逆时针为正）
#  - scale: 缩放比例
# 返回一个 2x3 的仿射变换矩阵
M = cv2.getRotationMatrix2D(
    (image_shape[1] // 2, image_shape[0] // 2),  # 图像中心点 (cx, cy)
    angle,  # 旋转角度
    scale,  # 缩放比例
)

# 4️⃣ 应用旋转矩阵进行图像仿射变换
# cv2.warpAffine(src, M, dsize[, dst[, flags[, borderMode[, borderValue]]]])
#  - src: 输入图像
#  - M: 2x3 仿射变换矩阵
#  - dsize: 输出图像大小 (宽, 高)
#  - flags: 插值方式
#       cv2.INTER_NEAREST（最近邻）
#       cv2.INTER_LINEAR（双线性，常用）
#       cv2.INTER_CUBIC（三次插值）
#  - borderMode: 边界填充方式
#       cv2.BORDER_CONSTANT（固定颜色填充）
#       cv2.BORDER_REFLECT（镜像填充）
#       cv2.BORDER_WRAP（环绕填充）
#  - borderValue: 边界填充颜色（BGR），默认黑色 (0, 0, 0)
image_rotation = cv2.warpAffine(
    image_np,
    M,
    (image_shape[1], image_shape[0]),  # 输出大小：保持与原图一致
    flags=cv2.INTER_LINEAR,  # 双线性插值
    borderMode=cv2.BORDER_CONSTANT,  # 使用常数填充边界
    borderValue=(255, 255, 255),  # 边界填充为黑色
)

# 5️⃣ 显示旋转结果
cv2.imshow("image_rotation", image_rotation)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 🧭 一、什么是仿射变换（Affine Transformation）

仿射变换是一种保持直线与比例关系的二维空间线性变换。

#### 👉 也就是说：

- 图像中的直线经过变换后仍然是直线；
- 平行线依旧平行；
- 相对比例保持不变；
- 但角度、长度和面积可以改变。

#### 🧩 仿射变换的几何作用

仿射变换可实现：

- 平移（Translation）
- 旋转（Rotation）
- 缩放（Scaling）
- 翻转（Reflection）
- 错切（Shearing）


#### 📐 数学形式
### 📘 仿射变换（Affine Transformation）

仿射变换是一种线性几何变换，保持直线和平行性。

---

#### 一、通用表达式

$$
\begin{bmatrix}
x' \\
y'
\end{bmatrix}
=
\begin{bmatrix}
a_{11} & a_{12} \\
a_{21} & a_{22}
\end{bmatrix}
\begin{bmatrix}
x \\
y
\end{bmatrix}
+
\begin{bmatrix}
t_x \\
t_y
\end{bmatrix}
$$

---

#### 二、齐次坐标形式

$$
\begin{bmatrix}
x' \\
y' \\
1
\end{bmatrix}
=
\begin{bmatrix}
a_{11} & a_{12} & t_x \\
a_{21} & a_{22} & t_y \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix}
$$

---

#### 参数说明

| 参数 | 含义 |
|------|------|
| \( (x, y) \) | 原始像素坐标 |
| \( (x', y') \) | 变换后像素坐标 |
| \( a_{11}, a_{12}, a_{21}, a_{22} \) | 线性变换部分（旋转、缩放、错切） |
| \( t_x, t_y \) | 平移项 |

---

#### 三、旋转矩阵示例

仅考虑旋转（不平移）时：

$$
\begin{bmatrix}
x' \\
y'
\end{bmatrix}
=
\begin{bmatrix}
\cos \theta & -\sin \theta \\
\sin \theta & \cos \theta
\end{bmatrix}
\begin{bmatrix}
x \\
y
\end{bmatrix}
$$

若同时考虑缩放系数 \( scale \)：

$$
\begin{bmatrix}
x' \\
y'
\end{bmatrix}
=
scale \times
\begin{bmatrix}
\cos \theta & -\sin \theta \\
\sin \theta & \cos \theta
\end{bmatrix}
\begin{bmatrix}
x \\
y
\end{bmatrix}
$$

--- 


#### 📦 在 OpenCV 中的实现
```
 ┌────────────────────────────┐
 │  getAffineTransform()      │──► 计算通用仿射矩阵（根据三点）
 ├────────────────────────────┤
 │  getRotationMatrix2D()     │──► 生成旋转 + 缩放矩阵（快速方式）
 ├────────────────────────────┤
 │  warpAffine()              │──► 执行仿射变换（应用矩阵）
 └────────────────────────────┘

```

OpenCV 使用以下两种函数来执行仿射变换：

| 函数                                               | 用途                  |
| ------------------------------------------------ | ------------------- |
| `cv2.getAffineTransform(src_points, dst_points)` | 根据三点对应关系生成 2×3 仿射矩阵 |
| `cv2.getRotationMatrix2D(center, angle, scale)`  | 生成旋转 + 缩放矩阵         |
| `cv2.warpAffine(src, M, dsize, ...)`             | 根据仿射矩阵执行变换          |

# 🧭 OpenCV 仿射变换函数总览表

# 🧭 OpenCV 仿射变换函数总览表（含 LaTeX 公式）

| 函数 | 功能 | 参数 | 参数说明 | 返回值 / 数学表达式 | 应用场景 |
|------|------|------|-----------|----------------------|-----------|
| **`cv2.getAffineTransform(src_points, dst_points)`** | 根据三对点生成 2×3 仿射矩阵 | `src_points`<br>`dst_points` | - `src_points`：原图像中三点坐标（shape=(3,2)，float32）<br>- `dst_points`：目标图像中对应的三点坐标 | $$ M = \begin{bmatrix} a_{11} & a_{12} & t_x \\ a_{21} & a_{22} & t_y \end{bmatrix} $$ | - 图像对齐与几何校正<br>- 旋转、平移、缩放、错切<br>- 文档矫正（三点映射） |
| **`cv2.getRotationMatrix2D(center, angle, scale)`** | 快速生成旋转 + 缩放矩阵 | `center`<br>`angle`<br>`scale` | - `center`：旋转中心 (x, y)<br>- `angle`：旋转角度（逆时针为正，单位°）<br>- `scale`：缩放比例（>1放大，<1缩小） | $$ M = \begin{bmatrix} \alpha & \beta & (1-\alpha)x_c - \beta y_c \\ -\beta & \alpha & \beta x_c + (1-\alpha)y_c \end{bmatrix}, \quad \alpha = scale\cos\theta, \quad \beta = scale\sin\theta $$ | - 图像旋转与缩放<br>- 人脸对齐、姿态调整<br>- 动画旋转效果 |
| **`cv2.warpAffine(src, M, dsize[, flags[, borderMode[, borderValue]]])`** | 根据仿射矩阵执行图像变换 | `src`<br>`M`<br>`dsize`<br>`flags`<br>`borderMode`<br>`borderValue` | - `src`：输入图像<br>- `M`：仿射矩阵 (2×3)<br>- `dsize`：输出图像尺寸 (width, height)<br>- `flags`：插值方式（见下表）<br>- `borderMode`：边界模式<br>- `borderValue`：边界填充颜色 | $$ \begin{bmatrix} x' \\ y' \end{bmatrix} = M \cdot \begin{bmatrix} x \\ y \\ 1 \end{bmatrix} $$ | - 执行旋转、平移、缩放、错切<br>- 数据增强（Data Augmentation）<br>- 倾斜图像矫正 |
| **插值方式 (`flags`)** | - | - | - | - | - |
| `cv2.INTER_NEAREST` | 最近邻插值 | - | 快速但锯齿明显 | - | 二值图像、掩膜放缩 |
| `cv2.INTER_LINEAR` | 双线性插值（默认） | - | 平滑自然，速度适中 | - | 一般图像缩放与旋转 |
| `cv2.INTER_CUBIC` | 三次插值 | - | 高质量但慢，适合放大 | - | 高清影像重建 |
| `cv2.INTER_AREA` | 像素区域插值 | - | 缩小时最佳 | - | 图像缩小、压缩 |
| **边界填充模式 (`borderMode`)** | - | - | - | - | - |
| `cv2.BORDER_CONSTANT` | 固定颜色填充 | - | 用 `borderValue` 指定颜色填充 | - | 默认边界模式 |
| `cv2.BORDER_REFLECT` | 镜像填充 | - | 以边界为轴反射 | - | 保留边缘细节 |
| `cv2.BORDER_REPLICATE` | 重复边缘像素 | - | 复制边缘值扩展 | - | 防止黑边 |
| `cv2.BORDER_WRAP` | 环绕填充 | - | 首尾相接循环 | - | 特殊纹理变换 |


### 🎯 二、插值（Interpolation）是什么？

当图像发生旋转、缩放或错切时，原像素点不再对应整型坐标。
这时就需要“插值”来估算这些新坐标处的像素值。

#### 🧠 原理图理解

想象原图是一个像素点阵，新图中的某个点 (x', y') 通过变换对应到原图的 (x, y)。

但 (x, y) 通常是个小数，比如 (100.3, 52.7)。

#### 👉 我们要根据附近整数像素点的值，推算出 (x, y) 的颜色值，这就是“插值”。
#### ⚙️ OpenCV 常见的插值方式

#### ⚙️ OpenCV 常见的插值方式详解

#### ⚙️ OpenCV 常见的插值方式详解（增强理解版）

| 方法 | 参数 | 原理简述 | 视觉特点 | 推荐使用场景 |
|------|------|----------|-----------|---------------|
| **最近邻插值** | `cv2.INTER_NEAREST` | **思想：** 找到目标像素对应的原图坐标 `(x, y)`，直接取距离最近的一个原始像素的值。<br>**公式：** $$f(x',y') = f(\text{round}(x), \text{round}(y))$$<br>**直观理解：** 想象把原图像素放到格子里，目标点落在哪个格子，就直接“抄写”最近格子的颜色；不做平滑，像是“放大像素积木”。<br>**生活比喻：** 就像放大照片时，每个像素变成一个更大的方块。 | 🚀速度最快<br>❌锯齿明显 | 二值图像、掩膜、语义分割标注、实时预览 |
| **双线性插值** | `cv2.INTER_LINEAR` | **思想：** 利用目标点周围的 **4 个像素**（上下左右）做加权平均。<br>**公式：** $$f(x,y) = (1-a)(1-b)f(i,j) + a(1-b)f(i+1,j) + (1-a)b f(i,j+1) + ab f(i+1,j+1)$$ 其中 $$a=x-\lfloor x \rfloor, \ b=y-\lfloor y \rfloor$$<br>**直观理解：** 像在四个点之间“拉一块橡皮布”，目标点的值由四个角共同拉扯决定；越靠近哪个角，那个角的权重越大。<br>**生活比喻：** 就像混合四个颜色，离谁近就多加一点谁的颜色。 | ⚖️平滑自然<br>✅默认方式 | 一般图像缩放、旋转、几何变换 |
| **三次插值** | `cv2.INTER_CUBIC` | **思想：** 考虑 **16 个像素点 (4×4 邻域)**，使用三次多项式进行拟合，使像素值和导数都连续。<br>**核心思想：** 插值函数 $$P(x)=\sum_{i=-1}^{2}w_i(x)f(i)$$，其中权重 \( w_i(x) \) 为三次卷积核。<br>**直观理解：** 像画一条平滑的曲线穿过多个像素点，不仅考虑点的位置，还保证过渡自然、导数平滑。<br>**生活比喻：** 就像用柔软的钢丝在 16 个点之间穿线，最终形成一条光滑曲线。 | 💎高质量<br>🐢较慢 | 高清放大、照片修复、医学影像 |
| **Lanczos 插值** | `cv2.INTER_LANCZOS4` | **思想：** 使用带窗 sinc 函数作为权重核，考虑 **8×8 邻域** 像素。<br>**公式：** $$L(x)=\text{sinc}(x)\cdot\text{sinc}\left(\frac{x}{a}\right)$$ 通常 \(a=4\)。<br>**直观理解：** sinc 函数权重像海浪一样起伏衰减——越远的像素影响越小，近处像素主导结果。它在数学上近似于理想低通滤波。<br>**生活比喻：** 像是“水波传播”，目标点的颜色是周围多个像素波动共同干涉的结果。 | 🌟极高画质<br>🧮计算最重 | 高端图像放大、科研图像、打印输出 |
| **区域插值（面积重采样）** | `cv2.INTER_AREA` | **思想：** 对缩小图像时，每个目标像素覆盖原图一块区域，将该区域内所有像素取加权平均。<br>**公式：** $$f'(x',y') = \frac{1}{N}\sum_{(x,y)\in R(x',y')} f(x,y)$$ 其中 \(R(x',y')\) 是目标像素对应的原图区域。<br>**直观理解：** 把原图块“压缩”成一个像素，取其平均亮度；就像在缩小一张照片时，每一块区域的亮度都被“融合”为一个更大的平均值。<br>**生活比喻：** 就像把几滴颜料混在一起后平均上色——保证颜色过渡自然，不会失真。 | ✅缩小时最优<br>💨速度快 | 缩小图像、视频帧采样、生成缩略图 |

#### 🧱 OpenCV 边界填充模式详解（Border Modes）

| 填充模式 | 参数 | 原理简述 | 视觉特点 | 推荐使用场景 |
|-----------|--------|------------|--------------|----------------|
| **常数填充** | `cv2.BORDER_CONSTANT` | **思想：** 超出图像边界的像素以固定值填充（默认黑色）。<br>**公式：** $$I'(x,y)=\begin{cases}I(x,y), & 0\le x<w,\ 0\le y<h \\ C, & \text{otherwise}\end{cases}$$ 其中 \(C\) 为填充值。<br>**直观理解：** 超出原图范围的像素都被“包上一圈”指定颜色的边框。<br>**生活比喻：** 像在画布边缘贴上彩色胶带，空白处全部涂成同一颜色。 | 🚩最直观<br>🟩可自定义边框颜色 | 图像旋转、平移、投影时的默认填充；可用于指定背景色场景 |
| **复制边界像素** | `cv2.BORDER_REPLICATE` | **思想：** 超出边界的像素值取最近边缘像素的值。<br>**公式：** $$I'(x,y)=I(\text{clamp}(x,0,w-1),\ \text{clamp}(y,0,h-1))$$<br>**直观理解：** 边缘的最后一列像素被“拉伸”出去，形成延展。<br>**生活比喻：** 就像用手指把画的边缘颜色往外抹开。 | ⚙️自然平滑<br>无突兀边缘 | 滤波/卷积时保持边界稳定；模糊、锐化、卷积操作常用 |
| **镜像填充（不重复边界）** | `cv2.BORDER_REFLECT` | **思想：** 图像边界处的像素向外对称反射，但不重复边界像素。<br>**示意：** `[1,2,3] → …3,2,1,2,3,2,1…`<br>**公式：** $$I'(x,y)=I(|x|,|y|)$$（当索引越界时取绝对对称位置）<br>**直观理解：** 像在镜子里反射原图边缘，产生平滑对称延展。<br>**生活比喻：** 像在图像两侧放镜子，画面在镜中连续延伸。 | 🌈边界平滑<br>连续性强 | 图像滤波、卷积、卷积神经网络输入边界扩展 |
| **镜像填充（重复边界）** | `cv2.BORDER_REFLECT_101`<br>或 `cv2.BORDER_DEFAULT` | **思想：** 与 `REFLECT` 类似，但会重复边界像素一次。<br>**示意：** `[1,2,3] → …3,2,1,1,2,3,3,2,1…`<br>**直观理解：** 在反射镜像的同时，边界像素本身也参与复制。<br>**生活比喻：** 像把照片贴在镜子上边缘，照片那一行也被映出来一次。 | 🌟平滑过渡<br>边缘连续 | 卷积神经网络、滤波器常用（默认方式） |
| **环绕填充** | `cv2.BORDER_WRAP` | **思想：** 超出边界的像素从另一侧“环绕”取值。<br>**示意：** `[1,2,3,4] → …3,4,1,2,3,4,1,2…`<br>**公式：** $$I'(x,y)=I((x\mod w),(y\mod h))$$<br>**直观理解：** 图像像贴在一个圆筒上，超出右边就从左边回来。<br>**生活比喻：** 像游戏地图“无尽滚动”，从边界出去又从另一侧出现。 | 🔄周期性<br>适合连续纹理 | 拼接平铺纹理、周期图像处理、傅里叶分析 |
| **反射（边界值外延）** | `cv2.BORDER_REFLECT101` | **思想：** 类似 REFLECT，但边界元素不会重复，即从倒数第二个像素开始反射。<br>**示意：** `[1,2,3] → …3,2,1,2,3,2…`<br>**直观理解：** 反射后不会出现边界的双影。<br>**生活比喻：** 像波浪在边缘反弹时不叠加。 | ✨边界平滑<br>无重复像素 | 纹理无缝拼接、卷积边界修复 |
| **透明填充** | `cv2.BORDER_TRANSPARENT` | **思想：** 不对边界之外的区域进行任何填充，保持原始透明度或忽略该区域。<br>**直观理解：** 仅处理原图部分，超出区域不参与计算。<br>**生活比喻：** 就像只在画布中心绘制，边缘部分留白透明。 | ⚪空白边缘<br>透明效果 | 图像混合、alpha合成、图层叠加时 |
